### Ingestion de la carpeta "production_company.csv"

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","2024-12-30")
v_file_date = dbutils.widgets.get("p_file_date")

#### Paso 1 - Leer el archivo CSV utilizando "DataFrameReader" de Spark

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
productions_companies_schema = StructType(fields=[
    StructField("companyId", IntegerType(), True),
    StructField("companyName", StringType(), True)
    ])


In [0]:
productions_companies_df = spark.read \
    .schema(productions_companies_schema) \
    .csv(f"{bronze_folder_path}/{v_file_date}/production_company")

#### Paso 2 - Cambiar el nombre de las columnas segun lo "requerido"

In [0]:
productions_companies_renamed_df = productions_companies_df \
    .withColumnRenamed("companyId","company_id") \
    .withColumnRenamed("companyName","company_name")
    

#### Paso 3 - Agregar una columna "ingestion date" al dataframe

In [0]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import lit

In [0]:
productions_companies_final_df = add_ingestion_date(productions_companies_renamed_df) \
.withColumn("enviroment", lit(dbutils.widgets.get("p_enviroment")))\
.withColumn("file_date", lit(v_file_date))

#### Paso 4 - Escribir datos en Datalake en formato parquet

In [0]:
display(productions_companies_final_df)

In [0]:
#overwrite_partition(productions_companies_final_df,"movie_silver","productions_companies","file_date")
merge_condition = 'tgt.company_id = src.company_id AND tgt.file_date = src.file_date'
merge_delta_lake(productions_companies_final_df,'movie_silver','productions_companies',silver_folder_path,merge_condition,'file_date')

In [0]:
%sql
SELECT file_date, COUNT(1) 
FROM movie_silver.productions_companies
GROUP BY file_date;